In [1]:
model_name = "bi_lstm_attention"
card_type = "Diamonds"

In [2]:
import pandas as pd
import numpy as np
from chance_utils import compute_statistics
from matplotlib import pyplot as plt

In [3]:
possible_result = ["7","8","9","10","J","Q","K","A"]
all_cards = ['Spades','Diamonds','Hearts','Clubs']
n_outputs = 8
n_features = 8*4
sample_length = 365*5

In [4]:
dummies = pd.get_dummies(possible_result).T
dummies = dummies.reindex(index=possible_result,fill_value=0)
dummies = dummies.reset_index()
dummies

,index,0,1,2,3,4,5,6,7
0,7,1,0,0,0,0,0,0,0
1,8,0,1,0,0,0,0,0,0
2,9,0,0,1,0,0,0,0,0
3,10,0,0,0,1,0,0,0,0
4,J,0,0,0,0,1,0,0,0
5,Q,0,0,0,0,0,1,0,0
6,K,0,0,0,0,0,0,1,0
7,A,0,0,0,0,0,0,0,1


In [5]:
def replace_with_dummies(df):
    df = pd.merge(dummies,df,left_on='index',right_on="Spades",how="right").drop(columns=["Spades"])
    df = pd.merge(dummies,df,left_on='index',right_on="Diamonds",how="right",suffixes=("_Diamonds","")).drop(columns=["Diamonds","index_Diamonds"])
    df = pd.merge(dummies,df,left_on='index',right_on="Hearts",how="right",suffixes=("_Hearts","")).drop(columns=["Hearts","index_Hearts"])
    return pd.merge(dummies,df,left_on='index',right_on="Clubs",how="right",suffixes=("_Clubs","")).drop(columns=["index","Clubs","index_Clubs"])

    

In [6]:
test_chance = pd.read_csv("test_chance.csv",index_col=[0])
test_chance.Date = pd.to_datetime(test_chance.Date)

train_chance = pd.read_csv('train_chance.csv',index_col=[0])
train_chance.Date = pd.to_datetime(train_chance.Date)

In [7]:
test_chance = replace_with_dummies(test_chance)
train_chance = replace_with_dummies(train_chance)

/var/folders/3r/ygns0_h16pl1__pln5g3j8t80000gp/T/ipykernel_54391/3572030588.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'3', '5', '6', '2', '7', '1', '0', '4'} in the result is deprecated and will raise a MergeError in a future version.
  return pd.merge(dummies,df,left_on='index',right_on="Clubs",how="right",suffixes=("_Clubs","")).drop(columns=["index","Clubs","index_Clubs"])
/var/folders/3r/ygns0_h16pl1__pln5g3j8t80000gp/T/ipykernel_54391/3572030588.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'3', '5', '6', '2', '7', '1', '0', '4'} in the result is deprecated and will raise a MergeError in a future version.
  return pd.merge(dummies,df,left_on='index',right_on="Clubs",how="right",suffixes=("_Clubs","")).drop(columns=["index","Clubs","index_Clubs"])


In [8]:
# make the samples ready to tsfresh
spades_test_chance = test_chance.sort_values(by="Date").reset_index(drop=True)
spades_test_chance['id'] = 1

spades_train_chance = train_chance.sort_values(by="Date").reset_index(drop=True)
spades_train_chance['id'] = 1

spades_train_chance.tail()

,0_Clubs,1_Clubs,2_Clubs,3_Clubs,4_Clubs,5_Clubs,6_Clubs,7_Clubs,0,1,...,1,2,3,4,5,6,7,Date,lottoryNumber,id
29119,0,0,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,2017-12-07 00:00:00.035924,35924,1
29120,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,2017-12-07 00:00:00.035925,35925,1
29121,0,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,2017-12-07 00:00:00.035926,35926,1
29122,0,0,0,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,2017-12-07 00:00:00.035927,35927,1
29123,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,2017-12-07 00:00:00.035928,35928,1


In [ ]:
from tsfresh.utilities.dataframe_functions import roll_time_series

df_test_rolled = roll_time_series(spades_test_chance, column_id="id",column_sort="Date",max_timeshift=sample_length-1,min_timeshift=sample_length-1)
df_train_rolled = roll_time_series(spades_train_chance, column_id="id",column_sort="Date",max_timeshift=sample_length-1,min_timeshift=sample_length-1)
df_train_rolled.tail()

/Users/serlich/Documents/Notebooks/env/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling: 100%|██████████████████████████████████| 40/40 [00:24<00:00,  1.63it/s]
/Users/serlich/Documents/Notebooks/env/lib/python3.8/site-packages/tsfresh/utilities/dataframe_functions.py:520: UserWarning: Your time stamps are not uniformly sampled, which makes rolling nonsensical in some domains.
  warnings.warn(
Rolling:  25%|████████▌                         | 10/40 [00:58<04:21,  8.73s/it]

In [ ]:
test_labeling_df = test_chance.drop(columns=["Date"])
train_labeling_df = train_chance.drop(columns=["Date"])

labeling_df = pd.concat([test_labeling_df,train_labeling_df])
labeling_df.head()

In [ ]:
def make_a_sample(current_sample,labeling_df):
    
    current_id = current_sample['id'].values[0]
    final_label = None
    
    assert current_sample.shape[0] == sample_length, f"{current_id} is not shape of {sample_length}"

    lottey_num_predicting = (current_id[1].microsecond + 1)
    label = labeling_df[labeling_df.lottoryNumber == lottey_num_predicting]
    
    if len(label) == 1:
        final_label = label.values[0]

    sample = current_sample.drop(columns=['id','Date']).values

    return lottey_num_predicting,final_label,sample

In [ ]:
train = df_train_rolled.groupby("id").apply(lambda x:make_a_sample(x,labeling_df))

train = sorted(train.values,key=lambda x:x[0])
_,train_y,train_x = map(list, zip(*train))

In [ ]:
test = df_test_rolled.groupby("id").apply(lambda x:make_a_sample(x,labeling_df))

test = sorted(test.values,key=lambda x:x[0])
_,test_y,test_x = map(list, zip(*test))

In [ ]:
test_y = pd.DataFrame(test_y[:-1])
last_sample = test_x[-1]

test_x = test_x[:-1]
train_y = pd.DataFrame(train_y)

In [ ]:
test_x = np.stack(test_x)
train_x = np.stack(train_x)

In [ ]:
test_x = test_x[:,:,:-1]
train_x = train_x[:,:,:-1]
test_y = test_y.iloc[:,:-1]
train_y = train_y.iloc[:,:-1]

In [ ]:
assert test_x.shape[0] == test_y.shape[0] and train_x.shape[0] == train_y.shape[0]

In [ ]:
print(f"Test shape {test_x.shape}")
print(f"Train shape {train_x.shape}")

In [ ]:
test_y.iloc[:,:8].shape

In [ ]:
from keras import backend as K

class attention(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W = self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b = self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

In [ ]:
import tensorflow as tf

model = None
if model_name == "lstm":
    #
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(300, input_shape=(sample_length,n_features)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

elif model_name == "attention":
    #
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.LSTM(50, input_shape=(sample_length,n_features),return_sequences=True))
    model.add(attention())
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(50, activation='relu'))
    model.add(tf.keras.layers.Dense(100, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
elif model_name == "bi_lstm":
    #
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),input_shape=(sample_length,n_features),merge_mode="concat",return_sequences=True))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(200, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

elif model_name == "bi_lstm_attention":
    #
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50,return_sequences=True),input_shape=(sample_length,n_features),merge_mode="concat"))
    model.add(attention())
    #model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(200, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
elif model_name == "nn":
    #
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Flatten(input_shape=(sample_length,8)))
    model.add(tf.keras.layers.Dense(350, activation='relu'))
    model.add(tf.keras.layers.Dense(250, activation='relu'))
    model.add(tf.keras.layers.Dense(150, activation='relu'))
    model.add(tf.keras.layers.Dense(n_outputs, activation='softmax'))
    model.compile(optimizer="adam",loss='categorical_crossentropy', metrics=['accuracy'])
    
model.summary()

In [ ]:
card_type_index = all_cards.index(card_type)
start_index = card_type_index*8
end_index = (card_type_index+1)*8

if test_y.shape[1] > 8:
    test_y = test_y.iloc[:,start_index:end_index]
    train_y = train_y.iloc[:,start_index:end_index]
    
assert test_y.shape[1] == 8

In [ ]:
assert not np.isnan(train_x).any()
assert not np.isnan(train_y).any().any()
assert not np.isnan(test_x).any()
assert not np.isnan(test_y).any().any()

In [ ]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=10)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=5, min_lr=0.001)
# 
history = model.fit(train_x,train_y,validation_data=(test_x,test_y),batch_size=64,epochs=300,callbacks=[es,reduce_lr])

In [ ]:
model.save('weight1')

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
predictions = model.predict(test_x)


In [ ]:
def get_card_from_encoding(encoding):
    if hasattr(encoding,"values"):
        gambels = encoding.values.argmax(axis=1)
    else:
        gambels = encoding.argmax(axis=1)
    return np.vectorize(lambda x:possible_result[x])(gambels)

In [ ]:
predictions_cards = get_card_from_encoding(predictions)
true_cards = get_card_from_encoding(test_y)

compute_statistics(predictions_cards,true_cards)